# STEP 1: Install dependencies

In [2]:
!pip install --upgrade pip

  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
C:\Users\voora\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install -q crewai google-generativeai duckduckgo_search yfinance pandas beautifulsoup4 python-dotenv
!pip install -q crewai-tools


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# STEP 2: Imports

In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, LLM
import yfinance as yf
from duckduckgo_search import DDGS
from crewai.tools import BaseTool
# from pydotenv import load_dotenv

# Load GEMINI API key (replace with your key or set in .env)

In [ ]:
# from getpass import getpass
os.environ["OPENAI_API_KEY"] = "Add_Your_OpenAI_API_Key_Here"

# STEP 3: Setup Gemini LLM via CrewAI

In [10]:
llm = LLM(
    model="gpt-3.5-turbo",  # Ensure this model is valid and accessible
    api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.7
)

# STEP 4: Define Tools

In [11]:
class StockSearchTool(BaseTool):
    name: str = "StockNewsSearcher"
    description: str = "Search for the latest news and updates about a stock using DuckDuckGo"

    def _run(self, query: str) -> str:
        with DDGS() as ddgs:
            results = ddgs.text(query, max_results=2)  # Limit to 2 results to reduce output
            return "\n".join([r['body'] for r in results])

class YahooFinanceTool(BaseTool):
    name: str = "YahooFinanceFetcher"
    description: str = "Get the latest 1-month stock price history for a given ticker using yFinance"

    def _run(self, ticker: str) -> str:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1mo")
        return hist.tail(3).to_string()  # Limit to last 3 rows for concise output

# STEP 5: Define Agents

In [12]:
stock_analyst = Agent(
    role='Stock Analyst',
    goal='Analyze recent stock data and news',
    backstory='Expert in financial trends, macro indicators, and company performance',
    verbose=True,  # Keep agent verbose for debugging, but we'll adjust Crew verbose
    allow_delegation=False,
    llm=llm
)

report_writer = Agent(
    role='Report Generator',
    goal='Write investor-friendly summaries of stock analysis',
    backstory='Professional writer with expertise in finance reporting',
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# STEP 6: Create Tasks

In [13]:
search_tool = StockSearchTool()
finance_tool = YahooFinanceTool()

search_task = Task(
    description="Search latest news and updates about the stock 'AAPL' using DuckDuckGo.",
    expected_output="Summarized news highlights for Apple stock.",
    agent=stock_analyst,
    tools=[search_tool]
)

analysis_task = Task(
    description="Analyze Apple stock price trends using yFinance.",
    expected_output="Key trends and technical highlights for the past month.",
    agent=stock_analyst,
    tools=[finance_tool]
)

report_task = Task(
    description="Write a clean investor report using previous analysis and news insights.",
    expected_output="Concise report with market summary and investment outlook.",
    agent=report_writer
)

# STEP 7: Assemble the Crew

In [14]:
crew = Crew(
    agents=[stock_analyst, report_writer],
    tasks=[search_task, analysis_task, report_task],
    verbose=False  # Set to False to reduce rich console output and avoid RecursionError
)

# STEP 8: Run the Agent Crew

In [15]:
result = crew.kickoff()

# STEP 10: Output the result
print("\n📊 Final Stock Analysis Report:\n")
print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Stock Analyst                                                                                           │
│                                                                                                                 │
│  Task: Search latest news and updates about the stock 'AAPL' using DuckDuckGo.                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

C:\Users\voora\AppData\Local\Temp\ipykernel_51424\4166229341.py:6: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Stock Analyst                                                                                           │
│                                                                                                                 │
│  Thought: I need to search for the latest news and updates about the stock 'AAPL' using DuckDuckGo to gather    │
│  summarized news highlights.                                                                                    │
│                                                                                                                 │
│  Using Tool: StockNewsSearcher                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "query": "AAPL"                                                                                              │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                   Open        High         Low       Close    Volume  Dividends  Stock Splits   │
│  Date                                                                                                           │
│  2025-11-05 00:00:00-05:00  268.609985  271.700012  266.929993  270.140015  43683100        0.0           0.0   │
│  2025-11-06 00:00:00-05:00  267.890015  273.399994  267.890015  269.769989  51157400        0.0           0.0   │
│  2025-11-07 00:00:00-05:00  269.799988  272.290009  266.769989  268.470001  48203600        0.0           0.0   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Stock Analyst                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Date                   Open        High         Low       Close    Volume  Dividends  Stock Splits             │
│  2025-11-05 00:00:00-05:00  268.609985  271.700012  266.929993  270.140015  43683100        0.0           0.0   │
│  2025-11-06 00:00:00-05:00  267.890015  273.399994  267.890015  269.769989  51157400        0.0           0.0   │
│  2025-11-07 00:00:00-05:00  269.799988  272.290009  266.769989  268.470001  48203600        0.0           0.0   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Generator                                                                                        │
│                                                                                                                 │
│  Task: Write a clean investor report using previous analysis and news insights.                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Generator                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Market Summary:**                                                                                            │
│  Over the past few days, the stock of AAPL has shown some fluctuations in its price, opening at $268.61 on 5th  │
│  November, reaching a high of $273.40 on 6th November, and closing at $268.47 on 7th November. The trading      │
│  volumes have been relatively high, indicating significant investor interest in the stock.                      │
│                                                                                                                 │
│  **Investment Outlook:**                                                                                        │
│  Based on the recent price movements and trading volumes, AAPL appears to be experiencing some volatility.      │
│  Investors should closely monitor any news or developments related to the company that could impact its stock   │
│  performance. Additionally, considering AAPL's historical performance and market position, long-term investors  │
│  may find potential value in holding the stock despite short-term fluctuations.                                 │
│                                                                                                                 │
│  It is essential for investors to conduct thorough research and consider their risk tolerance before making     │
│  any investment decisions related to AAPL.                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


📊 Final Stock Analysis Report:

**Market Summary:**
Over the past few days, the stock of AAPL has shown some fluctuations in its price, opening at $268.61 on 5th November, reaching a high of $273.40 on 6th November, and closing at $268.47 on 7th November. The trading volumes have been relatively high, indicating significant investor interest in the stock.

**Investment Outlook:**
Based on the recent price movements and trading volumes, AAPL appears to be experiencing some volatility. Investors should closely monitor any news or developments related to the company that could impact its stock performance. Additionally, considering AAPL's historical performance and market position, long-term investors may find potential value in holding the stock despite short-term fluctuations.

It is essential for investors to conduct thorough research and consider their risk tolerance before making any investment decisions related to AAPL.
